# Example: mesh a delineated watershed

Here we mesh the [Coweeta Hydrologic Laboratory](https://www.srs.fs.usda.gov/coweeta/) as an example of how to pull data in from default locations and generate a fully functional ATS mesh.

This might be the worst example to use to learn how to use Watershed Workflows.  But it is useful to demonstrate the breadth of problems this project was intended to solve.

This includes a range of datasets:

* NHD Plus for river network
* NRCS soils data for soil types
* NLCD for land cover/transpiration/rooting depths
* NED for elevation

In [1]:
%matplotlib osx

In [2]:
import os,sys
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm as pcm
import shapely
import logging

import workflow
import workflow.source_list
import workflow.ui
import workflow.colors
import workflow.condition
import workflow.mesh
import workflow.split_hucs

workflow.ui.setup_logging(1,None)

## Sources and setup

Next we set up the source watershed and coordinate system and all data sources for our mesh.  We will use the CRS that is included in the shapefile.

In [3]:
# specify the input shapefile and a hint as to what HUC it is in.
coweeta_shapefile = '../data/hydrologic_units/others/Coweeta/coweeta_basin.shp'
hint = '0601'  # hint: HUC 4 containing this shape.  
               # This is necessary to avoid downloading all HUCs to search for this shape
simplify = 100 # length scale to target average edge

logging.info("")
logging.info("Meshing shape: {}".format(coweeta_shapefile))
logging.info("="*30)

# get the shape and crs of the shape
crs, coweeta = workflow.get_split_form_shapes(coweeta_shapefile)

2020-05-13 17:22:31,646 - root - INFO: 
2020-05-13 17:22:31,648 - root - INFO: Meshing shape: ../data/hydrologic_units/others/Coweeta/coweeta_basin.shp
2020-05-13 17:22:31,650 - root - INFO: ==============================
2020-05-13 17:22:31,651 - root - INFO: 
2020-05-13 17:22:31,652 - root - INFO: Preprocessing Shapes
2020-05-13 17:22:31,653 - root - INFO: ------------------------------
2020-05-13 17:22:31,656 - root - INFO: loading file: "../data/hydrologic_units/others/Coweeta/coweeta_basin.shp"
/Users/uec/codes/anaconda/3/envs/ats_meshing_20190719/lib/python3.7/site-packages/fiona/collection.py:336: FionaDeprecationWarning: Collection slicing is deprecated and will be disabled in a future version.
  return self.session.__getitem__(item)


A wide range of data sources are available; here we use the defaults except for using NHD Plus for watershed boundaries and hydrography (the default is NHD, which is lower resolution and therefore smaller download sizes).

In [4]:
# set up a dictionary of source objects
sources = workflow.source_list.get_default_sources()
sources['HUC'] = workflow.source_list.huc_sources['NHD Plus']
sources['hydrography'] = workflow.source_list.hydrography_sources['NHD Plus']
workflow.source_list.log_sources(sources)

2020-05-13 17:22:31,736 - root - INFO: Using sources:
2020-05-13 17:22:31,737 - root - INFO: --------------
2020-05-13 17:22:31,740 - root - INFO: HUC: National Hydrography Dataset Plus High Resolution (NHDPlus HR)
2020-05-13 17:22:31,744 - root - INFO: hydrography: National Hydrography Dataset Plus High Resolution (NHDPlus HR)
2020-05-13 17:22:31,747 - root - INFO: DEM: National Elevation Dataset (NED)
2020-05-13 17:22:31,749 - root - INFO: soil type: National Resources Conservation Service Soil Survey (NRCS Soils)
2020-05-13 17:22:31,751 - root - INFO: land cover: National Land Cover Database (NLCD) Layer: NLCD_2016_Land_Cover_L48
2020-05-13 17:22:31,751 - root - INFO: soil thickness: None
2020-05-13 17:22:31,752 - root - INFO: meteorology: DayMet 1km


## Generate the surface mesh

First we'll generate the flattened, 2D triangulation, which builds on hydrography data.  Then we download a digital elevation map from the National Elevation Dataset, and extrude that 2D triangulation to a 3D surface mesh based on interpolation between pixels of the DEM.

In [5]:
rivers = False
if rivers:
    # download/collect the river network within that shape's bounds
    _, reaches = workflow.get_reaches(sources['hydrography'], hint, 
                                      coweeta.exterior().bounds, crs)
    # simplify and prune rivers not IN the shape, constructing a tree-like data structure
    # for the river network
    rivers = workflow.simplify_and_prune(coweeta, reaches, simplify=simplify, cut_intersections=True)

else:
    rivers = list()
    workflow.split_hucs.simplify(coweeta, simplify)



In [6]:
# form a triangulation on the shape + river network

# triangulation refinement:
# Refine triangles if their area (in m^2) is greater than A(d), where d is the 
# distance from the triangle centroid to the nearest stream.
# A(d) is a piecewise linear function -- A = A0 if d <= d0, A = A1 if d >= d1, and
# linearly interpolates between the two endpoints.
d0 = 100; d1 = 500
A0 = 1000; A1 = 5000
#A0 = 500; A1 = 2500
#A0 = 100; A1 = 500

# Refine triangles if they get too acute
min_angle = 32 # degrees

# make 2D mesh
#mesh_points2, mesh_tris, d = workflow.triangulate(coweeta, rivers, 
#                                               refine_distance=[d0,A0,d1,A1],
#                                               refine_min_angle=min_angle,
#                                               enforce_delaunay=True,
#                                               diagnostics=True)
mesh_points2, mesh_tris, d = workflow.triangulate(coweeta, rivers,
                                                 refine_max_area=100000,
                                                 enforce_delaunay=True,
                                                 diagnostics=True)

2020-05-13 17:22:31,816 - root - INFO: 
2020-05-13 17:22:31,818 - root - INFO: Meshing
2020-05-13 17:22:31,823 - root - INFO: ------------------------------
2020-05-13 17:22:31,825 - root - INFO: Triangulating...
2020-05-13 17:22:31,838 - root - INFO:    27 points and 27 facets
2020-05-13 17:22:31,839 - root - INFO:  checking graph consistency
2020-05-13 17:22:31,841 - root - INFO:  building graph data structures
2020-05-13 17:22:31,843 - root - INFO:  triangle.build...
2020-05-13 17:22:31,850 - root - INFO:   ...built: 153 mesh points and 256 triangles
2020-05-13 17:22:31,851 - root - INFO: Plotting triangulation diagnostics
2020-05-13 17:22:31,869 - root - INFO:   min area = 24107.691467285156


In [7]:
# get a raster for the elevation map
dem_profile, dem = workflow.get_raster_on_shape(sources['DEM'], coweeta.exterior(), crs)

# elevate the triangle nodes to the dem
mesh_points3 = workflow.elevate(mesh_points2, crs, dem, dem_profile)

2020-05-13 17:22:32,060 - root - INFO: 
2020-05-13 17:22:32,061 - root - INFO: Preprocessing Raster
2020-05-13 17:22:32,061 - root - INFO: ------------------------------
2020-05-13 17:22:32,062 - root - INFO: collecting raster
2020-05-13 17:22:32,064 - root - INFO: Collecting DEMs to tile bounds: [-83.48845037186388, 35.01734099944037, -83.41165773504302, 35.08381933600275]
2020-05-13 17:22:32,066 - root - INFO:   Need:
2020-05-13 17:22:32,067 - root - INFO:     /Users/uec/research/water/data/watershed-workflow/data-master/dem/USGS_NED_1as_n36_w084.img
2020-05-13 17:22:32,124 - root - INFO: 
2020-05-13 17:22:32,126 - root - INFO: Elevating Triangulation to DEM
2020-05-13 17:22:32,128 - root - INFO: ------------------------------


Plotting the resulting mesh can be done in a variety of ways, including both 3D plots and mapview.  We show both here, but hereafter use mapview plots as they are a bit clearer (if not so flashy)...

In [8]:
figsize = (6,6)
figsize_3d = (8,6)

In [9]:
# plot the resulting surface mesh
fig = plt.figure(figsize=figsize_3d)
ax = workflow.plot.get_ax('3d', fig, window=[0.0,0.2,1,0.8])
cax = fig.add_axes([0.23,0.18,0.58,0.03])

mp = ax.plot_trisurf(mesh_points3[:,0], mesh_points3[:,1], mesh_points3[:,2], 
                     triangles=mesh_tris, cmap='viridis', 
                     edgecolor=(0,0,0,.2), linewidth=0.5)
cb = fig.colorbar(mp, orientation="horizontal", cax=cax)

rivers_2 = [np.array(r.xy) for riv in rivers for r in riv]
rivers_e = [workflow.values_from_raster(r.transpose(), crs, dem, dem_profile) 
               for r in rivers_2]
rivers_l3 = [np.array([i[0], i[1], j]).transpose() 
               for i,j in zip(rivers_2, rivers_e)]
for r in rivers_l3:
    ax.plot(r[:,0]+1, r[:,1], r[:,2]+10, color='red', linewidth=3)

t = cax.set_title('elevation [m]')
ax.view_init(55,0)
ax.set_xticklabels(list())
ax.set_yticklabels(list())

#fig.savefig('coweeta_dem_3d')

[]

In [10]:
# plot the resulting surface mesh
fig = plt.figure(figsize=figsize)
ax = workflow.plot.get_ax(crs, fig)

mp = workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, 
                                 color='elevation', edgecolor='gray', linewidth=0.2)
cbar = fig.colorbar(mp, orientation="horizontal", pad=0.05)
workflow.plot.hucs(coweeta, crs, ax=ax, color='k', linewidth=1)
workflow.plot.rivers(rivers, crs, ax=ax, color='red', linewidth=1)
ax.set_aspect('equal', 'datalim')
ax.set_title('elevation [m]')
cbar.ax.set_title('elevation [m]')
#fig.savefig('coweeta_dem')

Text(0.5, 1.0, 'elevation [m]')

In [11]:
# construct the 2D mesh
m2 = workflow.mesh.Mesh2D(mesh_points3.copy(), list(mesh_tris))

In [12]:
# hydrologically condition the mesh
workflow.condition.condition(m2)

# plot the change between the two meshes
diff = np.copy(mesh_points3)
diff[:,2] = m2.points[:,2] - mesh_points3[:,2] 
print("max diff = ", np.abs(diff[:,2]).max())
fig, ax = workflow.plot.get_ax(crs, figsize=figsize)
workflow.plot.triangulation(diff, m2.conn, crs, color='elevation', edgecolors='gray', 
                            linewidth=0.2, ax=ax)
ax.set_title('conditioned dz')
plt.show()

max diff =  59.41045465459604


In [13]:
# contruct the dual mesh points, connectivity
dual_points2, dual_conn, dual_from_primary = m2.to_dual()

# elevate the dual
dual_points3 = workflow.elevate(dual_points2, crs, dem, dem_profile)

# construct the dual mesh
m2_dual = workflow.mesh.Mesh2D(dual_points3.copy(), dual_conn)

# plot the resulting surface mesh
fig = plt.figure(figsize=figsize)
ax = workflow.plot.get_ax(crs, fig)

#import matplotlib.cm
primal_elev = np.array([mesh_points3[i,2] for i in dual_from_primary])
#elev_cmap = workflow.colors.cm_mapper(primal_elev.min(), primal_elev.max(), matplotlib.cm.viridis)
#m2_dual.plot(color=np.array([elev_cmap(e) for e in primal_elev]), ax=ax)

workflow.plot.mesh(m2_dual, crs, ax=ax, color=primal_elev, facecolor='color', edgecolor='gray', linewidth=0.2)
cbar = fig.colorbar(mp, orientation="horizontal", pad=0.05)
workflow.plot.hucs(coweeta, crs, ax=ax, color='k', linewidth=1)
workflow.plot.rivers(rivers, crs, ax=ax, color='red', linewidth=1)
ax.set_aspect('equal', 'datalim')
ax.set_title('elevation [m]')
cbar.ax.set_title('elevation [m]')

2020-05-13 17:22:32,800 - root - INFO: Constructing Mesh2D as dual of a triangulation.
2020-05-13 17:22:32,802 - root - INFO: -- confirming triangulation (note, not checking delaunay, buyer beware)
2020-05-13 17:22:32,803 - root - INFO: -- computing primary boundary edges
2020-05-13 17:22:32,804 - root - INFO:      n_primal_cell = 256, n_boundary_edges = 48, n_dual_nodes = 352
2020-05-13 17:22:32,805 - root - INFO: -- computing dual nodes
2020-05-13 17:22:32,812 - root - INFO:     added 256 tri centroid nodes
2020-05-13 17:22:32,814 - root - INFO:     added 48 boundary nodes
2020-05-13 17:22:32,815 - root - INFO: -- Finding duplicates and ordering conn_cell_to_node
2020-05-13 17:22:32,838 - root - INFO: -- removing duplicate nodes
2020-05-13 17:22:32,841 - root - INFO: 
2020-05-13 17:22:32,842 - root - INFO: Elevating Triangulation to DEM
2020-05-13 17:22:32,843 - root - INFO: ------------------------------


Text(0.5, 1.0, 'elevation [m]')

## Surface properties

Meshes interact with data to provide forcing, parameters, and more in the actual simulation.  Specifically, we need vegetation type on the surface to provide information about transpiration and subsurface structure to provide information about water retention curves, etc.

We'll start by downloading and collecting land cover from the NLCD dataset, and generate sets for each land cover type that cover the surface.  Likely these will be some combination of grass, deciduous forest, coniferous forest, and mixed.

In [14]:
# download the NLCD raster
lc_profile, lc_raster = workflow.get_raster_on_shape(sources['land cover'], 
                                                     coweeta.exterior(), crs)

# resample the raster to the triangles
lc = workflow.values_from_raster(m2_dual.centroids(), crs, lc_raster, lc_profile)

# what land cover types did we get?
logging.info('Found land cover dtypes: {}'.format(lc.dtype))
logging.info('Found land cover types: {}'.format(set(lc)))


2020-05-13 17:22:33,239 - root - INFO: 
2020-05-13 17:22:33,240 - root - INFO: Preprocessing Raster
2020-05-13 17:22:33,241 - root - INFO: ------------------------------
2020-05-13 17:22:33,243 - root - INFO: collecting raster
2020-05-13 17:22:33,250 - root - INFO: CRS: PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],TOWGS84[0,0,0,-0,-0,-0,0],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1]]
2020-05-13 17:22:33,290 - root - INFO: Found land cover dtypes: uint8
2020-05-13 17:22:33,292 - root - INFO: Found land cover types: {41, 42, 43, 81

In [15]:
# plot the NLCD data

# -- get the NLCD colormap which uses official NLCD colors and labels
nlcd_indices, nlcd_cmap, nlcd_norm, nlcd_ticks, nlcd_labels = \
                workflow.colors.generate_nlcd_colormap(lc)

nlcd_labels_fw = []
for label in nlcd_labels:
    label_fw = label
    if len(label) > 15:
        if ' ' in label:
            lsplit = label.split()
            if len(lsplit) == 2:
                label_fw = '\n'.join(lsplit)
            elif len(lsplit) == 4:
                label_fw = '\n'.join([' '.join(lsplit[0:2]),
                                      ' '.join(lsplit[2:])])
            elif len(lsplit) == 3:
                if len(lsplit[0]) > len(lsplit[-1]):
                    label_fw = '\n'.join([lsplit[0],
                                          ' '.join(lsplit[1:])])
                else:
                    label_fw = '\n'.join([' '.join(lsplit[:-1]),
                                          lsplit[-1]])
    nlcd_labels_fw.append(label_fw)

fig = plt.figure(figsize=figsize)
ax = workflow.plot.get_ax(crs, fig)

polys = workflow.plot.mesh(m2_dual, crs, ax=ax, color=lc, cmap=nlcd_cmap, norm=nlcd_norm, edgecolor='none', 
                                     facecolor='color', linewidth=0.5)
mp = pcm.ScalarMappable(norm=nlcd_norm, cmap=nlcd_cmap)
cb = fig.colorbar(mp)
cb.set_ticks(nlcd_ticks)
cb.set_ticklabels(nlcd_labels_fw)
ax.set_title("land cover index")
fig.savefig('coweeta_nlcd')

## Subsurface properties

Get soil structure from SSURGO

In [16]:
# download the NRCS soils data as shapes and project it onto the mesh
import workflow.sources.manager_nrcs
import matplotlib.cm

# -- download the shapes
target_bounds = coweeta.exterior().bounds
_, soil_survey = workflow.get_shapes(sources['soil type'], target_bounds, crs)

# -- log the bounds targetted and found
logging.info('target bounds: {}'.format(target_bounds))
logging.info('shape union bounds: {}'.format(
    shapely.ops.cascaded_union(soil_survey).bounds))

# -- determine the NRCS mukey for each soil unit; this uniquely identifies soil 
#    properties
soil_ids = np.array([shp.properties['id'] for shp in soil_survey], np.int32)

# -- color a raster by the polygons (this makes identifying a triangle's value much 
#    more efficient)
soil_color_raster, soil_color_profile, img_bounds = \
            workflow.color_raster_from_shapes(target_bounds, 10, soil_survey,
                                              soil_ids, crs)

# -- resample the raster to the triangles
soil_color = workflow.values_from_raster(m2_dual.centroids(), crs, 
                                         soil_color_raster, soil_color_profile)

2020-05-13 17:22:33,758 - root - INFO: 
2020-05-13 17:22:33,759 - root - INFO: Preprocessing Shapes
2020-05-13 17:22:33,760 - root - INFO: ------------------------------
2020-05-13 17:22:33,762 - root - INFO:   Using filename: /Users/uec/research/water/data/watershed-workflow/data-master/soil_survey/soil_survey_shape_-83.4790_35.0269_-83.4208_35.0743.gml
2020-05-13 17:22:33,911 - root - INFO:   Found 460 shapes.
2020-05-13 17:22:33,911 - root - INFO:   and crs: {'init': 'epsg:4326'}
2020-05-13 17:22:34,042 - root - INFO: target bounds: (273971.0911428096, 3878839.6361173145, 279140.9150949494, 3883953.7853134344)
2020-05-13 17:22:34,653 - root - INFO: shape union bounds: (272780.3245135138, 3877673.165081467, 281292.5015333445, 3887703.7251368817)
2020-05-13 17:22:34,654 - root - INFO: Coloring shapes onto raster:
2020-05-13 17:22:34,655 - root - INFO:   target_bounds = (273971.0911428096, 3878839.6361173145, 279140.9150949494, 3883953.7853134344)
2020-05-13 17:22:34,656 - root - INFO:

In [17]:
# plot the soil data
fig = plt.figure(figsize=figsize)
ax = workflow.plot.get_ax(crs, fig)

mp = workflow.plot.mesh(m2_dual, crs, ax=ax, facecolor='color',
                                 linewidth=0.5, color=soil_color, cmap='copper')
ax.set_title('soil type index')

fig.savefig('coweeta_soils')

## Mesh extrusion

Given the surface mesh and material IDs on both the surface and subsurface, we can extrude the surface mesh in the vertical to make a 3D mesh.

In [18]:
# layer extrusion
# -- data structures needed for extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []
z = 0.0

# -- soil layer --
#  top 6 m
#  5 cm initial top cell
#  10 cells
#  expanding dz, growing with depth
ncells = 9
dz = 0.05
layer_dz = 4

tele = workflow.mesh.telescope_factor(ncells, dz, layer_dz)
print("Got telescoping factor: {}".format(tele))
for i in range(ncells):
    layer_types.append('constant')
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(soil_color)
    z += dz
    dz *= tele
    
# one more 2m layer makes 6m
dz = 2.0
layer_types.append('constant')
layer_data.append(dz)
layer_ncells.append(1)
layer_mat_ids.append(soil_color)
z += dz

# -- geologic layer --
# keep going for 2m cells until we hit the bottom of
# the domain
layer_types.append("constant")
layer_data.append(40 - z) # depth of bottom of domain is 40 m
layer_ncells.append(int(round(layer_data[-1] / dz)))
layer_mat_ids.append(999*np.ones_like(soil_color))

# print the summary
workflow.mesh.Mesh3D.summarize_extrusion(layer_types, layer_data, 
                                            layer_ncells, layer_mat_ids)

2020-05-13 17:22:35,666 - root - INFO: Cell summary:
2020-05-13 17:22:35,667 - root - INFO: ------------------------------------------------------------
2020-05-13 17:22:35,669 - root - INFO: l_id	| c_id	|mat_id	| dz		| z_top
2020-05-13 17:22:35,670 - root - INFO: ------------------------------------------------------------
2020-05-13 17:22:35,671 - root - INFO:  00 	| 00 	| 545806 	|   0.050000 	|   0.000000
2020-05-13 17:22:35,672 - root - INFO:  01 	| 01 	| 545806 	|   0.075796 	|   0.050000
2020-05-13 17:22:35,674 - root - INFO:  02 	| 02 	| 545806 	|   0.114899 	|   0.125796
2020-05-13 17:22:35,675 - root - INFO:  03 	| 03 	| 545806 	|   0.174177 	|   0.240695
2020-05-13 17:22:35,676 - root - INFO:  04 	| 04 	| 545806 	|   0.264036 	|   0.414872
2020-05-13 17:22:35,676 - root - INFO:  05 	| 05 	| 545806 	|   0.400255 	|   0.678908
2020-05-13 17:22:35,678 - root - INFO:  06 	| 06 	| 545806 	|   0.606751 	|   1.079163
2020-05-13 17:22:35,681 - root - INFO:  07 	| 07 	| 545806 	|   0

Got telescoping factor: 1.515910144611108


In [19]:
# extrude
m3 = workflow.mesh.Mesh3D.extruded_Mesh2D(m2_dual, layer_types, layer_data, 
                                             layer_ncells, layer_mat_ids)

In [20]:
# add back on land cover side sets
surf_ss = m3.side_sets[1]

for index, name in zip(nlcd_indices, nlcd_labels):
    where = np.where(lc == index)[0]
    ss = workflow.mesh.SideSet(name, int(index), 
                            [surf_ss.elem_list[w] for w in where],
                            [surf_ss.side_list[w] for w in where])        
    m3.side_sets.append(ss)

In [21]:
# save to disk
try:
    os.remove('coweeta_basin_dual.exo')
except FileNotFoundError:
    pass
m3.write_exodus('coweeta_basin_dual.exo')


You are using exodus.py v 1.13 (seacas-beta), a python wrapper of some of the exodus library.

Copyright (c) 2013, 2014, 2015, 2016, 2017, 2018, 2019 National Technology &
Engineering Solutions of Sandia, LLC (NTESS).  Under the terms of
Contract DE-NA0003525 with NTESS, the U.S. Government retains certain
rights in this software.

Opening exodus file: coweeta_basin_dual.exo
Closing exodus file: coweeta_basin_dual.exo


In [22]:
import jigsawpy
from matplotlib.collections import LineCollection

In [23]:
opts = jigsawpy.jigsaw_jig_t()
geom = jigsawpy.jigsaw_msh_t()
mesh = jigsawpy.jigsaw_msh_t()

geom.mshID = "euclidean-mesh"
geom.ndims = +2
geom.vert2 = np.array([(tuple(c[0:2]), 0) for c in m2.coords], 
        dtype=geom.VERT2_t)

geom.edge2 = np.array([(e,0) for e in m2.edges()],
        dtype=geom.EDGE2_t)

opts.mesh_dims = +2                 # 2-dim. simplexes
jigsawpy.lib.jigsaw(opts, geom, mesh)


In [24]:
mesh.edge2['index']

array([[ 641, 1089],
       [ 536,  883],
       [ 364,  531],
       ...,
       [ 191,  848],
       [ 679,  875],
       [ 707, 1365]], dtype=int32)

In [ ]:
def plot(jmesh):
    fig = plt.figure()
    ax = fig.add_subplot(111)  ax.triplot(jmesh.point['coord'][:,0], jmesh.point['coord'][:,1], jmesh.tria3['index'])
    plt.show()
    #segs = np.array([jmesh.point['coord'][e] for e in jmesh.tria3['index']])
    #print(segs.shape)
    #lc = LineCollection(segs)

    #ax.add_collection(lc)
    #ax.set_xlim(segs[:,:,0].min(), segs[:,:,0].max())
    #ax.set_ylim(segs[:,:,1].min(), segs[:,:,1].max())

                                                                        

In [ ]:
import workflow.triangulation

# alternative -- triangulate with jigsaw
segments = list(coweeta.segments)
segments = segments + list(workflow.river_tree.forest_to_list(rivers))
print(len(segments))
nodes_edges = workflow.triangulation.NodesEdges(segments)
nodes_edges.check(tol=1)


In [ ]:
len(nodes_edges.edges)

In [ ]:
opts = jigsawpy.jigsaw_jig_t()
geom = jigsawpy.jigsaw_msh_t()
mesh = jigsawpy.jigsaw_msh_t()

geom.mshID = "euclidean-mesh"
geom.ndims = +2
geom.vert2 = np.array([(tuple(n), 0) for n in nodes_edges.nodes], 
        dtype=geom.VERT2_t)

geom.edge2 = np.array([(e,0) for e in nodes_edges.edges],
        dtype=geom.EDGE2_t)


opts.mesh_kern = "delfront"         # DELAUNAY kernel
opts.hfun_hmax = 0.05               # push HFUN limits
opts.mesh_dims = +2                 # 2-dim. simplexes
opts.optm_qlim = +.95
opts.mesh_top1 = True               # for sharp feat's
opts.geom_feat = True



jigsawpy.lib.jigsaw(opts, geom, mesh)

#scr2 = jigsawpy.triscr2(            # "quality" metric
#        mesh.point["coord"],
#        mesh.tria3["index"])

#jigsawpy.savevtk("coweeta_jigsaw.vtk", mesh)

In [ ]:
plot(mesh)

In [ ]:
print([k for k in mesh.__dict__])

In [ ]:
print(mesh.slope)

In [ ]:
import collections
import scipy.spatial

# get coords in 2D
coords = m2.coords[:,0:2]
eps = 0.1

point = collections.namedtuple('Point', ['x', 'y'])

def circumcenter(coord1,coord2,coord3):  # {{{
    p1 = point(*coord1)
    p2 = point(*coord2)
    p3 = point(*coord3)

    d = 2 * (p1.x * (p2.y - p3.y) + p2.x *
                (p3.y - p1.y) + p3.x * (p1.y - p2.y))

    xv = ((p1.x**2 + p1.y**2) * (p2.y - p3.y) + (p2.x**2 + p2.y**2)
          * (p3.y - p1.y) + (p3.x**2 + p3.y**2) * (p1.y - p2.y)) / d
    yv = ((p1.x**2 + p1.y**2) * (p3.x - p2.x) + (p2.x**2 + p2.y**2)
          * (p1.x - p3.x) + (p3.x**2 + p3.y**2) * (p2.x - p1.x)) / d

    return (xv, yv)

# collection of all dual nodes given by:
# - primal nodes on the boundary
# - primal edge midpoits on the boundary (note this length is the same as primal nodes on the boundary)
# - primal cell circumcenters?
boundary_edges = m2.boundary_edges()
logging.info("computed boundary edges")
n_tot = len(m2.conn) + 2*len(boundary_edges)
logging.info("  n_primal_cell = {}, n_boundary_edges = {}, n_total_dual_nodes = "
             "{}".format(len(m2.conn), len(boundary_edges), n_tot))
dual_nodes = np.zeros((n_tot, 2), 'd')
dual_cells = [list() for i in range(len(coords))]

is_boundary = np.zeros(len(dual_cells), 'i')

#
# Loop over all primal cells (triangles), adding the circumcenter 
# as a dual node and sticking that node in three dual cells rooted
# at the three primal nodes.
#  
i_dual_node = 0
for j, c in enumerate(m2.conn):
    dual_nodes[i_dual_node][:] = circumcenter(coords[c[0]], coords[c[1]], coords[c[2]])
    dual_cells[c[0]].append(i_dual_node)
    dual_cells[c[1]].append(i_dual_node)
    dual_cells[c[2]].append(i_dual_node)
    i_dual_node += 1
  
logging.info("added tri centroid nodes ({})".format(i_dual_node)) 

#
# Loop over the boundary, adding both the primal nodes as dual nodes 
# and the edge midpoints as dual nodes.
#
# Add the primal node and two midpoints on either side to the list 
# of dual nodes in the cell "rooted at" the primal node.
#
for i, e in enumerate(boundary_edges):
    # add the primal node always
    my_dual_node = i_dual_node
    dual_nodes[i_dual_node][:] = coords[e[0]]
    i_dual_node += 1

    my_cell = list()

    # stick in the previous midpoint node
    if i is 0:
        ugh_point = e[0]
        my_cell.append(-1)
    else:
        my_cell.append(prev_midp_n)

    # stick in the next midpoint node, if it isn't too close
    next_midp = (coords[e[0]][:] + coords[e[1]][:])/2.
    next_midp_n = i_dual_node
    for n in dual_cells[e[0]]:
        if np.linalg.norm(dual_nodes[n] - next_midp) < eps:
            next_midp_n = n
            break
    if next_midp_n == i_dual_node:
        dual_nodes[i_dual_node][:] = next_midp
        i_dual_node += 1
    
    my_cell.append(next_midp_n)
    my_cell.append(my_dual_node)
    dual_cells[e[0]].extend(my_cell)
    is_boundary[e[0]] = 1
    prev_midp_n = next_midp_n
    
assert(dual_cells[ugh_point][-3] == -1)
dual_cells[ugh_point][-3] = prev_midp_n
logging.info("added boundary nodes") 




#
# Now every dual cell has a list of nodes, and the dual coordinates are done.
#
# Must still order the nodes in each dual cell
#
for i in range(len(dual_cells)):
    c = dual_cells[i]
    
    if is_boundary[i]:
        # may not be convex -- triangulate
        c_orig = c[:]
        c0 = c[-1]
        cdn = c[-3]
        cup = c[-2]
        
        cell_coords = np.array([dual_nodes[i] for i in c[:-1]]) - dual_nodes[c0]
        angle = np.array([np.arctan2(cell_coords[j,1], cell_coords[j,0]) for j in range(len(cell_coords))])
        order = np.argsort(angle)
        c = [c[j] for j in order]
        up_i = c.index(cup)
        dn_i = c.index(cdn)
        
        if dn_i == (up_i + 1)%len(c):
            cn = c[dn_i:] + c[0:dn_i]
        elif up_i == (dn_i + 1)%len(c):
            cn = c[up_i:] + c[0:up_i]
        else:
            print("Uh oh borked geom: up_i = {}, dn_i = {}, c = {}".format(up_i, dn_i, c))
            fig = plt.figure()
            ax = fig.add_subplot(111)

            cc_sorted = np.array([cell_coords[k] for k in order]) + dual_nodes[c0]
            ax.plot(cc_sorted[:,0], cc_sorted[:,1], 'k-x')            
            ax.scatter(dual_nodes[c0,0], dual_nodes[c0,1], color='r')
            ax.scatter(dual_nodes[cup,0], dual_nodes[cup,1], color='m')
            ax.scatter(dual_nodes[cdn,0], dual_nodes[cdn,1], color='b')
            plt.show()
            
            
            fig = plt.figure(figsize=figsize)
            ax = workflow.plot.get_ax(crs, fig)

            mp = workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, 
                                 color='elevation', edgecolor='white', linewidth=0.4)
            cbar = fig.colorbar(mp, orientation="horizontal", pad=0.05)
            workflow.plot.hucs(coweeta, crs, ax=ax, color='k', linewidth=1)
            workflow.plot.shply([shapely.geometry.LineString(cc_sorted),], crs, ax=ax, color='red', linewidth=1)
            ax.set_aspect('equal', 'datalim')
            
            raise RuntimeError('uh oh borked geom')
        
        #print('Boundary: c_orig = {}, c = {}, cn = {}'.format(c_orig, c, cn))
        for k in range(len(cn)-1):
            if k == 0:
                dual_cells[i] = [c0, cn[k+1], cn[k]]
            else:
                dual_cells.append([c0, cn[k+1], cn[k]])
                
    else:   
        cell_coords = np.array([dual_nodes[i] for i in c])
        cell_centroid = cell_coords.mean(axis=0)
        cell_coords = cell_coords - cell_centroid

        angle = np.array([np.arctan2(cell_coords[j,1], cell_coords[j,0]) for j in range(len(cell_coords))])
        order = np.argsort(angle)
        dual_cells[i] = [c[j] for j in order]
    
logging.info("sorted nodes")
 
dual_points3 = workflow.elevate(dual_nodes[:,0:2], crs, dem, dem_profile)
logging.info("elevated dual")



In [ ]:
m2_dual = workflow.extrude.Mesh2D(dual_points3.copy(), dual_cells)
m2_dual.plot()